In [ ]:
# Copyright 2024 Daniel Franzen and Jan Disselhoff
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
# This notebook contains our winning submission to the ARC Prize 2024 Kaggle competition,
# scoring 53.5 points on the private evaluation set.
# the ARChitects (Daniel Franzen and Jan Disselhoff)

In [1]:
# THIS IS IF WE WANT TO RUN ON MORE THAN ONE GPU
NUM_GPUS=1
import os
os.environ["WANDB_DISABLED"] = "true"

if NUM_GPUS>1:
    from common_stuff import *
    if not os.path.exists(os.path.join(tmp_dir, 'unsloth_installed')):  # unsloth offline install - https://stackoverflow.com/a/51646354
        !pip uninstall --yes torch accelerate
        !pip install --no-index --find-links=/kaggle/input/unsloth-2024-9-post4/wheelhouse unsloth
        #!pip uninstall --yes accelerate fastai torch torchaudio transformers
        #!pip install --no-index --find-links=/kaggle/input/unsloth-2024-10-7/wheelhouse unsloth  # do not use grad_acc_fix - trains very slow
        #!sed -i 's/if ((post_check - pre_check) >= 1).sum() > 1:/if False:/g' /opt/conda/lib/python3.10/site-packages/unsloth/models/llama.py
        # fix delay bug in get_statistics()
        !sed -i 's/^def get_statistics():/def get_statistics():\n if False:/g' /opt/conda/lib/python3.10/site-packages/unsloth/models/_utils.py
        # fix faulty unsloth multi-gpu detection
        !sed -i "s/raise RuntimeError('Unsloth currently does not support multi GPU setups - but we are working on it!')/pass/g" /opt/conda/lib/python3.10/site-packages/unsloth/tokenizer_utils.py /opt/conda/lib/python3.10/site-packages/unsloth/models/llama.py /opt/conda/lib/python3.10/site-packages/unsloth/models/vision.py
        os.makedirs(os.path.join(tmp_dir, 'unsloth_installed'), exist_ok=True)
        print('Unsloth installed & patched.')

    for gpu in [0, 1]: 
        signal_path = f'{model_temp_storage}_gpu{gpu}_done'
        if os.path.exists(signal_path): os.rmdir(signal_path)

    if arc_test_set.is_fake:  # cleanup? (for debugging)
        #!rm -R /kaggle/temp/finetuned_model*
        #!rm -R /kaggle/temp/inference_outputs
        #!rm -R /kaggle/temp/inference_scoring
        #!ls /kaggle/temp
        pass

In [3]:
# If using the --bg flag this will execute asynchronously 
# (you will then need to uncomment the await cell)
# %%python --bg --proc train_proc0
from common_stuff import *
start_training(gpu=0, augment=False)

*** Load challanges from '/home/nikolai/Desktop/Uni/1_software/arc-prize-2024-winning-solution/local_notebooks/../data/arc-prize-2024/arc-agi_evaluation_challenges.json'...

We are running: False
*** Load base model and tokenizer from './temp/da-fr--Mistral-NeMo-Minitron-8B-ARChitects-Full-bnb-4bit/transformers/default/1'...
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/nikolai/Desktop/Uni/1_software/arc-prize-2024-winning-solution/env_arc24_winning/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using

==((====))==  Unsloth 2025.1.6: Fast Mistral patching. Transformers: 4.48.1.
   \\   /|    GPU: NVIDIA GeForce RTX 3090. Max memory: 23.669 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
*** Create new peft model...


...

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 200 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 2
\        /    Total batch size = 4 | Total steps = 50
 "-____-"     Number of trainable parameters = 188,723,200


Step,Training Loss
10,0.029800
20,0.039400
30,0.024800
40,0.041500
50,0.030400


*** -> Training took 385.6371 seconds.
*** Saving model/tokenizer to './temp/finetuned_model_gpu0'...


/home/nikolai/Desktop/Uni/1_software/arc-prize-2024-winning-solution/env_arc24_winning/lib/python3.10/site-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


*** GPU: NVIDIA GeForce RTX 3090, used 9.46 / 23.7 GB.


In [8]:
# %%python --bg --proc train_proc1
# from common_stuff import *
# start_training(gpu=1)

In [1]:
# %%python --bg --proc infer_proc0
from common_stuff import *
start_inference(gpu=0, augment=False)

*** Load challanges from '/home/nikolai/Desktop/Uni/1_software/arc-prize-2024-winning-solution/local_notebooks/../data/arc-prize-2024/arc-agi_evaluation_challenges.json'...
*** Load base model and tokenizer from './temp/finetuned_model_gpu0'...
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/nikolai/Desktop/Uni/1_software/arc-prize-2024-winning-solution/env_arc24_winning/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.1.6: Fast Mistral patching. Transformers: 4.48.1.
   \\   /|    GPU: NVIDIA GeForce RTX 3090. Max memory: 23.669 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.1.6 patched 40 layers with 40 QKV layers, 40 O layers and 40 MLP layers.


Note we are skipping augmentation
*** Reducing task size to max. 8192 tokens (7260 input + 932 generated)...
100%|██████████| 208/208 [00:00<00:00, 513.52it/s] 
*** Load stored data...
*** Start inference run...
100%|██████████| 200/200 [54:39<00:00, 16.40s/it]
*** Completed inference run.
calculate augmented scores: 100%|██████████| 202/202 [20:39<00:00,  6.14s/it]
*** GPU: NVIDIA GeForce RTX 3090, used 7.28 / 23.7 GB.


In [10]:
# %%python --bg --proc infer_proc1
# from common_stuff import *
# start_inference(gpu=1)

In [4]:
# This is only needed if we launch the pythcon process as python --bg
# proc_exit_codes = await wait_for_subprocesses(train_proc0, train_proc1, infer_proc0, infer_proc1, print_output=True or arc_test_set.is_fake)
# print(f'*** Subprocesses exit codes: {proc_exit_codes}')
# assert all(x==0 for x in proc_exit_codes)

In [2]:
# write submission
from common_stuff import *
with RemapCudaOOM():
    model, formatter, dataset = None, MyFormatter(), None
    decoder = Decoder(formatter, arc_test_set.split_multi_replies(), n_guesses=2, frac_score=True).from_store(infer_params['store'])
    if use_aug_score or arc_test_set.is_fake: decoder.calc_augmented_scores(model=model, store=score_temp_storage, **aug_score_params)
    submission = arc_test_set.get_submission(decoder.run_selection_algo(submission_select_algo))
    with open('submission.json', 'w') as f: json.dump(submission, f)
    if arc_test_set.is_fake:
        decoder.benchmark_selection_algos(selection_algorithms)
        with open('submission.json') as f: reload_submission = json.load(f)
        print('*** Reload score:', arc_test_set.validate_submission(reload_submission))

calculate augmented scores: 100%|██████████| 202/202 [00:01<00:00, 154.56it/s]
*** Generating submission for 202 outputs...
